In [1]:
#Before using spark, need to install and set up spark, Hadoop and Jdk locally on your laptop


from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("etl") \
    .config("spark.master", "local") \
    .config("spark.jars", "F:/UniPrj/Big Data Project/postgresql-42.2.16.jar") \
    .getOrCreate()

#Load Amazon Data into Spark DataFrame
#1 load product segment - Video Segment --> download dataset tsv file
file_path = "F:/UniPrj/Big Data Project/Datasets/amazon_reviews_us_Video_v1_00.tsv"
df = spark.read.option("encoding", "UTF-8").csv(file_path, sep="\t", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [2]:
#Create DataFrame - Perform Preliminary Cleaning

#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

(380604, 15)


In [3]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   49033728|R1P1G5KZ05H6RD|6302503213|The Night They Sa...|           Video|          5|            0|          0|                Y|
|   17857748|R106N066IUN8ZV|B000059PET|Hamlet / Kline, N...|           Video|          5|            0|          0|                Y|
|   25551507| R7WTAA1S5O7D9|0788812807|Nascar Dual Power...|           Video|          4|            0|          0|                Y|
|   21025041|R32HFMVWLYOYJK|6302509939|The Man From U.N....|           Video|          5|            0|          0|                Y|
|   40943563| RWT3H6HBVAL6G|B00JENS2BI|Playboy Video Par...|  

In [4]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(95732, 9)


In [5]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   49033728|R1P1G5KZ05H6RD|6302503213|The Night They Sa...|           Video|          5|            0|          0|
|   17857748|R106N066IUN8ZV|B000059PET|Hamlet / Kline, N...|           Video|          5|            0|          0|
|   25551507| R7WTAA1S5O7D9|0788812807|Nascar Dual Power...|           Video|          4|            0|          0|
|   21025041|R32HFMVWLYOYJK|6302509939|The Man From U.N....|           Video|          5|            0|          0|
|   17013969|R1S3T3GWUGQTW7|6305761302|Cabaret Balkan - ...|           Video|          5|            0|          0|
|   47611685|R3R0QYHA800REE|6300157555|Wrinkles:in Need ...|           V

In [6]:
#Create Analysis-Specific DFs/Tables
#Perform Analysis-Specific Transforms
#Segmentation Analysis DF

# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   49033728|           Video|
|   17857748|           Video|
|   25551507|           Video|
|   21025041|           Video|
|   17013969|           Video|
|   47611685|           Video|
|   10747909|           Video|
|     126341|           Video|
|   40676812|           Video|
|   19863533|           Video|
|    1989554|           Video|
|   14885930|           Video|
|   24203443|           Video|
|   14885930|           Video|
|   41133700|           Video|
|   46580399|           Video|
|   19863533|           Video|
|   25551507|           Video|
|   25016862|           Video|
|    1989554|           Video|
+-----------+----------------+
only showing top 20 rows



In [7]:
#Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'videos')
#2c Check results
segment_df.show()

+-----------+------+
|customer_id|videos|
+-----------+------+
|   50851028|     1|
|   20686161|     2|
|   15191478|     2|
|    2202715|     1|
|   48333068|     1|
|   22910908|     2|
|   38463562|     1|
|   26902200|     1|
|   33236947|     1|
|   40127650|     1|
|   41244127|     3|
|   22968117|     1|
|   25472255|     1|
|    3160356|     1|
|   18785199|     2|
|   19840529|     1|
|    1655605|     1|
|   26298325|     1|
|   25705446|     1|
|   29337875|     1|
+-----------+------+
only showing top 20 rows



In [8]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- videos: long (nullable = false)

65517


In [9]:
#Note: Column Name in df.sort('name' ...) must align with Column name from step 2
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort("videos", ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())


+-----------+------+
|customer_id|videos|
+-----------+------+
|   18116317|   286|
|   43022286|   272|
|   49778371|   138|
|   43921586|   136|
|   14315565|   118|
|   52287429|    96|
|   49355567|    91|
|   51010646|    84|
|   15536614|    82|
|   38724344|    82|
|   45087308|    81|
|   24833576|    76|
|   28169415|    59|
|   34736206|    59|
|   53089838|    57|
|   50010538|    54|
|   12200981|    53|
|   49422747|    52|
|   44038210|    52|
|   47370694|    52|
+-----------+------+
only showing top 20 rows

65517


In [10]:
#Segmentation ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}


In [11]:
#Note: table name in table='name_segment' must align with table name in Postgres

# Write segment table to Postgres/RDS
# 15 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='videos_segment', mode=mode, properties=config)     

In [12]:
# Run Queries in Postgres to Confirm Segment ETL
# Check Row Count of Segment Table - SELECT COUNT(*) FROM videos_segment;

# Check 10 Rows of Segment Table - SELECT * FROM videos_segment LIMIT(10);


# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


In [13]:
# Apriori Analysis DF
# Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   49033728|R1P1G5KZ05H6RD|6302503213|
|   17857748|R106N066IUN8ZV|B000059PET|
|   25551507| R7WTAA1S5O7D9|0788812807|
|   21025041|R32HFMVWLYOYJK|6302509939|
|   17013969|R1S3T3GWUGQTW7|6305761302|
|   47611685|R3R0QYHA800REE|6300157555|
|   10747909| RGORN81H45NI7|B000SXQ5US|
|     126341|R1CNYN4ABNOJSU|B00008F22G|
|   40676812|R2DW06821PMN40|6303453961|
|   19863533|R1CS8AMA8B0VBJ|6302453178|
|    1989554| R3XP0G8P2BOTP|B000XPQ18W|
|   14885930|R3NK7014K996PF|0800113055|
|   24203443| ROIQI6ZW6UVL9|6300133338|
|   14885930| RECPIN4UZI76Z|6304119054|
|   41133700|R3FE5KO613PMVI|B000006CPD|
|   46580399|R2EC74PS45RAOR|189218608X|
|   19863533|R3FZ5C78PP7ZD7|6302754348|
|   25551507|R2Z9SKOUI9C9IR|6302320402|
|   25016862|R37DDS6DLZTK60|B000EQJFWY|
|    1989554|R3NPKF41EJ8U44|6301801601|
+-----------+--------------+----------+
only showing top 20 rows



In [14]:
#Prelim Apriori ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [15]:
#Note: table name in table='segment_apriori' must align with table name in Postgres


# Write segment_apriori table to RDS
# 16 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='videos_apriori', mode=mode, properties=config)
     

In [16]:
# Run Queries in Postgres to Confirm Apriori ETL
# Check Row Count of Apriori Table - SELECT COUNT(*) FROM videos_apriori;

# Check 10 Rows of Apriori Table - SELECT * FROM videos_apriori LIMIT(10);


# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
